# 03 Make Network Version 02 (with Assignment Group Updates)

In [3]:
import os
import sys
import yaml
from subprocess import Popen
import pickle

import pandas as pd
import geopandas as gpd
import numpy as np


from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import WranglerLogger
from network_wrangler import ProjectCard
from network_wrangler import Scenario

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import metcouncil

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

### Remote I/O and Parameters

In [6]:
root_dir = os.path.join('D:/', 'david_ory_github', 'client_met_council_network')
# root_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'client_met_council_network')
input_dir = os.path.join(root_dir, 'network_standard')
output_dir = input_dir
card_dir = os.path.join(root_dir, 'project_cards')
lasso_dir = os.path.join('C:\\', 'Users', 'david.ory', 'Documents', 'GitHub', 'lasso')
# lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'lasso')

In [7]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2021-06-17 13:58:35, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\lasso
2021-06-17 13:58:35, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\lasso


### Read Version 01 Scenario

In [8]:
version_01_pickle_file_name = os.path.join(input_dir, 'working_scenario_01.pickle')
v_01_scenario = pickle.load(open(version_01_pickle_file_name, 'rb'))

### Apply Project Cards

In [10]:
v_02_scenario = Scenario.create_scenario(
    base_scenario={"road_net": v_01_scenario.road_net, "transit_net": v_01_scenario.transit_net},
    card_directory = card_dir,
    tags = ['Network Cleanup AsgnGrp'],
    validate_project_cards=False)

2021-06-17 14:13:32, INFO: Creating Scenario
2021-06-17 14:13:32, INFO: Creating Scenario


c:\users\david.ory\documents\github\network_wrangler\network_wrangler\projectcard.py:114: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  attribute_dictionary = yaml.load(_yaml)


In [11]:
v_02_scenario.apply_all_projects()

2021-06-17 14:13:34, INFO: Applying Network Cleanup Remove Link
2021-06-17 14:13:34, INFO: Applying Network Cleanup Remove Link
2021-06-17 14:13:34, INFO: Applying Network Cleanup Remove Link
2021-06-17 14:13:34, INFO: Applying Network Cleanup Remove Link
2021-06-17 14:13:34, INFO: Applying Project to Roadway Network: Network Cleanup Remove Link
2021-06-17 14:13:34, INFO: Applying Project to Roadway Network: Network Cleanup Remove Link
2021-06-17 14:13:34, INFO: Applying Network Cleanup 1 AsgnGrp 7
2021-06-17 14:13:34, INFO: Applying Network Cleanup 1 AsgnGrp 7
2021-06-17 14:13:34, INFO: Applying Network Cleanup 1 AsgnGrp 7
2021-06-17 14:13:34, INFO: Applying Network Cleanup 1 AsgnGrp 7
2021-06-17 14:13:34, INFO: Applying Project to Roadway Network: Network Cleanup 1 AsgnGrp 7
2021-06-17 14:13:34, INFO: Applying Project to Roadway Network: Network Cleanup 1 AsgnGrp 7


C:\Users\david.ory\anaconda3\envs\lasso_x\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


2021-06-17 14:13:35, INFO: Applying Network Cleanup 1 AsgnGrp 6
2021-06-17 14:13:35, INFO: Applying Network Cleanup 1 AsgnGrp 6
2021-06-17 14:13:35, INFO: Applying Network Cleanup 1 AsgnGrp 6
2021-06-17 14:13:35, INFO: Applying Network Cleanup 1 AsgnGrp 6
2021-06-17 14:13:35, INFO: Applying Project to Roadway Network: Network Cleanup 1 AsgnGrp 6
2021-06-17 14:13:35, INFO: Applying Project to Roadway Network: Network Cleanup 1 AsgnGrp 6
2021-06-17 14:13:36, INFO: Applying Network Cleanup 1 AsgnGrp 50
2021-06-17 14:13:36, INFO: Applying Network Cleanup 1 AsgnGrp 50
2021-06-17 14:13:36, INFO: Applying Network Cleanup 1 AsgnGrp 50
2021-06-17 14:13:36, INFO: Applying Network Cleanup 1 AsgnGrp 50
2021-06-17 14:13:36, INFO: Applying Project to Roadway Network: Network Cleanup 1 AsgnGrp 50
2021-06-17 14:13:36, INFO: Applying Project to Roadway Network: Network Cleanup 1 AsgnGrp 50
2021-06-17 14:13:36, INFO: Applying Network Cleanup 1 AsgnGrp 4
2021-06-17 14:13:36, INFO: Applying Network Cleanu

In [12]:
v_02_scenario.applied_projects

['Network Cleanup Remove Link',
 'Network Cleanup 1 AsgnGrp 7',
 'Network Cleanup 1 AsgnGrp 6',
 'Network Cleanup 1 AsgnGrp 50',
 'Network Cleanup 1 AsgnGrp 4',
 'Network Cleanup 1 AsgnGrp 2',
 'Network Cleanup 1 AsgnGrp 15',
 'Network Cleanup 1 AsgnGrp 1']

### Make Travel Model Network

In [13]:
m_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = v_02_scenario.road_net, 
    parameters = parameters)

2021-06-17 14:21:09, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\lasso
2021-06-17 14:21:09, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\lasso


In [14]:
m_net.roadway_standard_to_met_council_network()

2021-06-17 14:21:09, INFO: Renaming roadway attributes to be consistent with what metcouncil's model is expecting
2021-06-17 14:21:09, INFO: Renaming roadway attributes to be consistent with what metcouncil's model is expecting
2021-06-17 14:21:09, INFO: Creating managed lane network.
2021-06-17 14:21:09, INFO: Creating managed lane network.
2021-06-17 14:21:09, INFO: Creating network with duplicated managed lanes
2021-06-17 14:21:09, INFO: Creating network with duplicated managed lanes


C:\Users\david.ory\anaconda3\envs\lasso_x\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


2021-06-17 14:21:50, INFO: Creating calculated roadway variables.
2021-06-17 14:21:50, INFO: Creating calculated roadway variables.
2021-06-17 14:21:50, INFO: Area Type Variable 'area_type' already in network. Returning without overwriting.
2021-06-17 14:21:50, INFO: Area Type Variable 'area_type' already in network. Returning without overwriting.
2021-06-17 14:21:50, INFO: County Variable 'county' already in network. Returning without overwriting.
2021-06-17 14:21:50, INFO: County Variable 'county' already in network. Returning without overwriting.
2021-06-17 14:21:50, INFO: Calculating MPO as roadway network variable: mpo
2021-06-17 14:21:50, INFO: Calculating MPO as roadway network variable: mpo
2021-06-17 14:21:50, INFO: Finished calculating MPO variable: mpo
2021-06-17 14:21:50, INFO: Finished calculating MPO variable: mpo
2021-06-17 14:21:50, INFO: Adding Counts
2021-06-17 14:21:50, INFO: Adding Counts
2021-06-17 14:21:50, INFO: Adding Variable AADT using Shared Streets Reference

### Write to Disk

In [15]:
m_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, 'v02_links.txt'),
    output_node_txt = os.path.join(output_dir, 'v02_nodes.txt'),
    output_link_header_width_txt = os.path.join(output_dir, 'links_header_width.txt'),
    output_node_header_width_txt = os.path.join(output_dir, 'nodes_header_width.txt'),
    output_cube_network_script = os.path.join(output_dir, 'make_complete_network_from_fixed_width_file.s'),
)

2021-06-17 14:27:29, INFO: Starting fixed width conversion
2021-06-17 14:27:29, INFO: Starting fixed width conversion
2021-06-17 15:36:40, INFO: Writing out link database
2021-06-17 15:36:40, INFO: Writing out link database
2021-06-17 15:37:00, INFO: Writing out link header and width ----
2021-06-17 15:37:00, INFO: Writing out link header and width ----
2021-06-17 15:37:00, INFO: Starting fixed width conversion
2021-06-17 15:37:00, INFO: Starting fixed width conversion
2021-06-17 15:38:33, INFO: Writing out node database
2021-06-17 15:38:33, INFO: Writing out node database
2021-06-17 15:38:35, INFO: Writing out node header and width
2021-06-17 15:38:35, INFO: Writing out node header and width


In [16]:
standard_transit_net = StandardTransit.fromTransitNetwork(v_02_scenario.transit_net, parameters=parameters)
standard_transit_net.write_as_cube_lin(outpath = os.path.join(output_dir, "v02_transit.lin"))

2021-06-17 15:38:37, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\lasso
2021-06-17 15:38:37, INFO: Lasso base directory set as: C:\Users\david.ory\Documents\GitHub\lasso
2021-06-17 15:38:37, INFO: Converting GTFS Standard Properties to MetCouncil's Cube Standard
2021-06-17 15:38:37, INFO: Converting GTFS Standard Properties to MetCouncil's Cube Standard


In [18]:
scenario_filename = os.path.join(output_dir, 'working_scenario_02.pickle')
pickle.dump(v_02_scenario, open(scenario_filename, 'wb'))